In [ ]:
from astroquery.vizier import Vizier
from astroquery.mast import Catalogs
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.cosmology import Planck18
import numpy as np
import pandas as pd

# Use Planck18 cosmology
cosmo = Planck18

# Your targets
targets = [
    ("20h40m47.886s", "+72d52m56.378s", 0.0430),
    ("08h58m52.92s",  "+73d29m27.0s",   0.4790),
    ("21h12m10.760s", "+72d49m38.20s",  0.3005),
    ("04h45m38.64s",  "+70d18m26.6s",   0.2505),
    ("21h00m31.09s",  "+72d02m15.22s",  0.5100),
    ("11h51m07.52s",  "+71d41m44.3s",   0.2710),
    ("05h52m45.12s",  "+74d12m01.7s",   1.0000),
    ("20h20m08.92s",  "+70d47m33.96s",  0.3024),
    ("02h39m03.96s",  "+71d01m04.3s",   1.0000),
    ("20h50m28.59s",  "+73d54m00.0s",  0.0740),
    ("11h19m56.05s",  "+70d40m34.4s",   0.2870),
    ("22h23m53.94s",  "+73d01m33.26s",  1.0000),
]

targets = [
    ("20h40m47.886s", "+72d52m56.378s"),
    ("08h58m52.92s",  "+73d29m27.0s"),
    ("21h12m10.760s", "+72d49m38.20s"),
    ("04h45m38.64s",  "+70d18m26.6s"),
    ("21h00m31.09s",  "+72d02m15.22s"),
    ("11h51m07.52s",  "+71d41m44.3s"),
    ("05h52m45.12s",  "+74d12m01.7s"),
    ("20h20m08.92s",  "+70d47m33.96s"),
    ("02h39m03.96s",  "+71d01m04.3s"),
    ("20h50m28.59s",  "+73d54m00.0s"),
    ("11h19m56.05s",  "+70d40m34.4s"),
    ("22h23m53.94s",  "+73d01m33.26s"),
]

# Setup VizieR
v = Vizier(row_limit=10000)

# Function to calculate angular size from physical size
def physical_to_angular(size_kpc, z):
    """Convert physical size in kpc to angular size in arcsec at redshift z"""
    d_A = cosmo.angular_diameter_distance(z)
    # theta = size / distance (in radians), then convert to arcsec
    theta_rad = (size_kpc * u.kpc / d_A).decompose().value  # This gives radians
    theta_arcsec = theta_rad * u.rad.to(u.arcsec)
    return theta_arcsec

# Function to calculate impact parameter
def calc_impact_param(ra_gal, dec_gal, z_gal, ra_sight, dec_sight):
    """Calculate physical impact parameter in kpc"""
    c1 = SkyCoord(ra_sight, dec_sight, unit='deg')
    c2 = SkyCoord(ra_gal, dec_gal, unit='deg')
    sep = c1.separation(c2)  # Angular separation
    
    # Convert angular separation to physical at galaxy redshift
    d_A = cosmo.angular_diameter_distance(z_gal)
    b_kpc = (sep.radian * d_A).to(u.kpc).value
    return b_kpc

# Calculate search radii for all targets
print("Search radii for 100 kpc physical:")
for ra_str, dec_str, z_target in targets:
    radius_arcsec = physical_to_angular(100, z_target)
    print(f"z={z_target:.4f}: radius = {radius_arcsec:.1f} arcsec = {radius_arcsec/60:.2f} arcmin")

# Search each sightline
all_results = []

# Available catalogs at these declinations
catalogs_2df = ['VII/250']    # 2dFGRS
catalogs_wise = ['II/328/allwise']  # WISE All-Sky
catalogs_des = ['II/371']     # DES DR2 (if available via VizieR)

for i, (ra_str, dec_str, z_target) in enumerate(targets):
    coord = SkyCoord(ra_str, dec_str, frame='icrs')
    
    # Calculate search radius
    radius_arcsec = physical_to_angular(100, z_target)
    radius = radius_arcsec * u.arcsec
    
    print(f"\n{'='*60}")
    print(f"Target {i+1}: {ra_str}, {dec_str} at z={z_target:.3f}")
    print(f"Search radius: {radius_arcsec:.1f} arcsec ({radius_arcsec/60:.2f} arcmin)")
    
    # Dictionary to store results for this target
    target_results = {
        'target_ra': coord.ra.deg,
        'target_dec': coord.dec.deg,
        'target_z': z_target,
        'galaxies': []
    }
    
    # Query 2dFGRS (spectroscopic redshifts)
    try:
        result_2df = v.query_region(coord, radius=radius, catalog=catalogs_2df)
        if len(result_2df) > 0:
            df = result_2df[0].to_pandas()
            if 'z' in df.columns:
                # Filter by redshift (allowing for peculiar velocities ~1000 km/s)
                z_tolerance = 0.003 + 0.1 * z_target  # Larger tolerance at higher z
                df_filtered = df[(df['z'] > z_target - z_tolerance) & 
                               (df['z'] < z_target + z_tolerance)].copy()
                
                if len(df_filtered) > 0:
                    # Calculate impact parameters
                    df_filtered['impact_kpc'] = df_filtered.apply(
                        lambda row: calc_impact_param(
                            row['RAJ2000'], row['DEJ2000'], row['z'],
                            coord.ra.deg, coord.dec.deg
                        ), axis=1
                    )
                    
                    # Filter by impact parameter
                    df_filtered = df_filtered[df_filtered['impact_kpc'] <= 100]
                    
                    if len(df_filtered) > 0:
                        print(f"Found {len(df_filtered)} galaxies in 2dFGRS within 100 kpc")
                        for _, gal in df_filtered.iterrows():
                            target_results['galaxies'].append({
                                'catalog': '2dFGRS',
                                'ra': gal['RAJ2000'],
                                'dec': gal['DEJ2000'],
                                'z': gal['z'],
                                'z_type': 'spec',
                                'impact_kpc': gal['impact_kpc'],
                                'mag_b': gal.get('Bjmag', np.nan)
                            })
    except Exception as e:
        print(f"2dFGRS query failed: {e}")
    
    # For higher redshift targets, also query WISE
    if z_target > 0.2:
        try:
            result_wise = v.query_region(coord, radius=radius, catalog=catalogs_wise)
            if len(result_wise) > 0:
                df_wise = result_wise[0].to_pandas()
                print(f"Found {len(df_wise)} WISE sources (need cross-match for redshifts)")
        except Exception as e:
            print(f"WISE query failed: {e}")
    
    all_results.append(target_results)

# Summary
print(f"\n{'='*60}")
print("SUMMARY:")
for i, result in enumerate(all_results):
    n_gal = len(result['galaxies'])
    if n_gal > 0:
        print(f"\nTarget {i+1} (z={result['target_z']:.3f}): {n_gal} galaxies within 100 kpc")
        for gal in result['galaxies']:
            print(f"  - {gal['catalog']}: z={gal['z']:.4f}, impact={gal['impact_kpc']:.1f} kpc")
    else:
        print(f"\nTarget {i+1} (z={result['target_z']:.3f}): No galaxies found within 100 kpc")

Search radii for 100 kpc physical:
z=0.0430: radius = 114.1 arcsec = 1.90 arcmin
z=0.4790: radius = 16.3 arcsec = 0.27 arcmin
z=0.3005: radius = 21.7 arcsec = 0.36 arcmin
z=0.2505: radius = 24.7 arcsec = 0.41 arcmin
z=0.5100: radius = 15.7 arcsec = 0.26 arcmin
z=0.2710: radius = 23.4 arcsec = 0.39 arcmin
z=1.0000: radius = 12.1 arcsec = 0.20 arcmin
z=0.3024: radius = 21.6 arcsec = 0.36 arcmin
z=1.0000: radius = 12.1 arcsec = 0.20 arcmin
z=0.0740: radius = 68.8 arcsec = 1.15 arcmin
z=0.2870: radius = 22.4 arcsec = 0.37 arcmin
z=1.0000: radius = 12.1 arcsec = 0.20 arcmin

Target 1: 20h40m47.886s, +72d52m56.378s at z=0.043
Search radius: 114.1 arcsec (1.90 arcmin)

Target 2: 08h58m52.92s, +73d29m27.0s at z=0.479
Search radius: 16.3 arcsec (0.27 arcmin)
Found 2 WISE sources (need cross-match for redshifts)

Target 3: 21h12m10.760s, +72d49m38.20s at z=0.300
Search radius: 21.7 arcsec (0.36 arcmin)
Found 4 WISE sources (need cross-match for redshifts)

Target 4: 04h45m38.64s, +70d18m26.6s at

/Users/jakobfaber/.local/share/mamba/envs/chimedsa/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:2623: RuntimeWarning: invalid value encountered in do_format (vectorized)
  outputs = ufunc(*args, out=...)
/Users/jakobfaber/.local/share/mamba/envs/chimedsa/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:2623: RuntimeWarning: invalid value encountered in do_format (vectorized)
  outputs = ufunc(*args, out=...)
/Users/jakobfaber/.local/share/mamba/envs/chimedsa/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:2623: RuntimeWarning: invalid value encountered in do_format (vectorized)
  outputs = ufunc(*args, out=...)
/Users/jakobfaber/.local/share/mamba/envs/chimedsa/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:2623: RuntimeWarning: invalid value encountered in do_format (vectorized)
  outputs = ufunc(*args, out=...)
/Users/jakobfaber/.local/share/mamba/envs/chimedsa/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:262

In [4]:
connorfrb = SkyCoord(ra=" 01h33m47s", dec="+31d51m30.886s", frame='icrs')
print(connorfrb.ra.deg, connorfrb.dec.deg)

m33 = SkyCoord(ra="01h33m50.02s", dec="+30d39m36.7s", frame='icrs')
print(m33.ra.deg, m33.dec.deg)

23.445833333333333 31.858579444444445
23.458416666666665 30.660194444444443


In [58]:
def galaxies_within_impact(ra, dec, z_t, d_kpc=100,
                           catalog="J/MNRAS/416/2840"):   # 2M++ default
    """Return a DataFrame of galaxies within d_kpc and |Δv|≤900 km s⁻¹."""
    sight = SkyCoord(ra, dec, unit=(u.hourangle, u.deg))

    theta_max = ((d_kpc * u.kpc).to(u.Mpc) /
                 cosmo.angular_diameter_distance(z_t)) * u.rad
    radius = 1.5 * theta_max              # 50 % padding

    viz = Vizier(columns=["RAJ2000", "DEJ2000", "z", "e_z"])
    viz.ROW_LIMIT = -1
    tables = viz.query_region(sight, radius=radius, catalog=catalog)

    # ---------- NEW: gracefully handle an empty reply ----------
    if len(tables) == 0:              # nothing in this catalogue
        return pd.DataFrame()         # return an empty DF so concat still works
    tab = tables[0].to_pandas()
    # ------------------------------------------------------------

    # velocity window (≈900 km s⁻¹)
    mask = (abs(tab["z"] - z_t) < 0.003 * (1 + z_t))
    tab = tab[mask]

    # proper‐distance impact parameter
    sep = sight.separation(
        SkyCoord(tab["RAJ2000"], tab["DEJ2000"], unit="deg")
    )
    tab["impact_kpc"] = (
        sep * cosmo.angular_diameter_distance(z_t)
    ).to(u.kpc).value

    return tab[tab["impact_kpc"] <= d_kpc]


In [59]:
master = pd.concat([galaxies_within_impact(*t) for t in targets],
                   keys=range(len(targets)), names=["target"])

/Users/jakobfaber/.local/share/mamba/envs/chimedsa/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:2623: RuntimeWarning: invalid value encountered in do_format (vectorized)
  outputs = ufunc(*args, out=...)
/Users/jakobfaber/.local/share/mamba/envs/chimedsa/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:2623: RuntimeWarning: invalid value encountered in do_format (vectorized)
  outputs = ufunc(*args, out=...)
/Users/jakobfaber/.local/share/mamba/envs/chimedsa/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:2623: RuntimeWarning: invalid value encountered in do_format (vectorized)
  outputs = ufunc(*args, out=...)
/Users/jakobfaber/.local/share/mamba/envs/chimedsa/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:2623: RuntimeWarning: invalid value encountered in do_format (vectorized)
  outputs = ufunc(*args, out=...)
/Users/jakobfaber/.local/share/mamba/envs/chimedsa/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:262

In [9]:
import pandas as pd
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u
from astropy.constants import c as speed_of_light

def find_and_save_matches(targets, catalog_path):
    """
    Searches a local FITS catalog and saves matches to separate CSV files for each target.
    """
    print(f"Loading FITS catalog from: {catalog_path}")

    try:
        catalog = Table.read(catalog_path, hdu=1)
    except Exception as e:
        print(f"Error loading FITS catalog: {e}")
        return

    target_coords = [SkyCoord(ra, dec, frame='icrs') for ra, dec, z in targets]
    target_redshifts = [z for ra, dec, z in targets]
    
    # Initialize a dictionary to store matches
    target_matches = {}

    print("\n--- Starting Search ---")
    
    for gal_row in catalog:
        try:
            gal_ra = gal_row['RA']
            gal_dec = gal_row['DEC']
            gal_velocity_kms = float(gal_row['V'])
            gal_z = gal_velocity_kms / (speed_of_light.to('km/s').value)
        except (ValueError, TypeError, KeyError):
            continue

        if gal_z <= 0:
            continue

        gal_coord = SkyCoord(ra=gal_ra*u.deg, dec=gal_dec*u.deg, frame='icrs')

        for i, (tgt_coord, tgt_z) in enumerate(zip(target_coords, target_redshifts)):
            target_index = i + 1
            if gal_z >= tgt_z:
                continue

            angular_sep = tgt_coord.separation(gal_coord)
            D_A = cosmo.angular_diameter_distance(gal_z)
            max_angular_sep = (100 * u.kpc / D_A).to(u.deg, u.dimensionless_angles())

            if angular_sep <= max_angular_sep:
                impact_param = (angular_sep.to(u.rad).value * D_A).to(u.kpc)
                
                # --- THE FIX: Build the coordinate strings manually ---
                match_data = {
                    'sightline_ra_deg': f"{tgt_coord.ra.deg:.6f}",
                    'sightline_dec_deg': f"{tgt_coord.dec.deg:.6f}",
                    'sightline_z': f"{tgt_z:.4f}",
                    'galaxy_ra_deg': f"{gal_coord.ra.deg:.6f}",
                    'galaxy_dec_deg': f"{gal_coord.dec.deg:.6f}",
                    'galaxy_z': f"{gal_z:.4f}",
                    'angular_separation_deg': f"{angular_sep.to(u.deg).value:.4f}",
                    'impact_parameter_kpc': f"{impact_param.value:.2f}"
                }
                
                if target_index not in target_matches:
                    target_matches[target_index] = []
                target_matches[target_index].append(match_data)

    print("\n--- Search Complete ---")

    if not target_matches:
        print("\nNo matches found in the catalog.")
        return

    print("\nSaving matches to CSV files...")
    for target_index, matches in target_matches.items():
        df = pd.DataFrame(matches)
        filename = f"target_{target_index}_matches.csv"
        df.to_csv(filename, index=False)
        print(f"✅ Saved {len(matches)} matches for Target {target_index} to {filename}")


# --- Your Data ---
targets = [
    #("20h40m47.886s", "+72d52m56.378s", 0.0430), 
    ("08h58m52.92s",  "+73d29m27.0s",   0.4790),
    #("21h12m10.760s", "+72d49m38.20s",  0.3005), ("04h45m38.64s",  "+70d18m26.6s",   0.2505),
    #("21h00m31.09s",  "+72d02m15.22s",  0.5100), ("11h51m07.52s",  "+71d41m44.3s",   0.2710),
    #("05h52m45.12s",  "+74d12m01.7s",   1.0000), ("20h20m08.92s",  "+70d47m33.96s",  0.3024),
    #("02h39m03.96s",  "+71d01m04.3s",   1.0000), ("20h50m28.59s",  "+73d54m00.0s",  0.0740),
    #("11h19m56.05s",  "+70d40m34.4s",   0.2870), ("22h23m53.94s",  "+73d01m33.26s",  1.0000),
]

# --- RUN THE SEARCH AND SAVE ---
find_and_save_matches(
    targets, 
    '/Users/jakobfaber/Documents/research/caltech/ovro/dsa110/chime_dsa_codetections/FLITS/galaxies/2mrs_v240/catalog/2mrs_1175_done.fits'
)

Loading FITS catalog from: /Users/jakobfaber/Documents/research/caltech/ovro/dsa110/chime_dsa_codetections/FLITS/galaxies/2mrs_v240/catalog/2mrs_1175_done.fits

--- Starting Search ---



--- Search Complete ---

Saving matches to CSV files...
✅ Saved 2 matches for Target 1 to target_1_matches.csv


In [6]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import Planck18 as cosmo   # H0 = 67.66 km/s/Mpc

# ------------------------------------------------------------
# 1. Coordinates (ICRS)
# ------------------------------------------------------------
#gal = SkyCoord(ra=150.829758*u.deg, dec=68.733727*u.deg)
#los = SkyCoord(ra=134.720500*u.deg, dec=73.490833*u.deg)
	
	
los = SkyCoord(ra="177.7813333333", dec="+71.6956388889", unit='deg', frame='icrs')
gal = SkyCoord(ra="177.7811216239492", dec="+71.7097754067686", unit='deg', frame='icrs')
	
# ------------------------------------------------------------
# 2. Redshift of the foreground galaxy
# ------------------------------------------------------------
z_gal = 0.118448 # 0.00001000697

# ------------------------------------------------------------
# 3. Angular separation θ
# ------------------------------------------------------------
theta = gal.separation(los)            # Angle object (38.78°)

# ------------------------------------------------------------
# 4. Angular-diameter distance to the galaxy
# ------------------------------------------------------------
D_A = cosmo.angular_diameter_distance(z_gal)   # ≈ 44.3 kpc

# ------------------------------------------------------------
# 5. Impact parameter  b = θ · D_A
#    Tell Astropy that “rad” should be treated as dimensionless
# ------------------------------------------------------------
with u.set_enabled_equivalencies(u.dimensionless_angles()):
    b = (theta * D_A).to(u.kpc)

print(f"θ  = {theta.to(u.deg):.4f}")
print(f"D_A = {D_A.to(u.kpc):.3f}")
print(f"b   = {b:.2f}")


θ  = 0.0141 deg
D_A = 456014.014 kpc
b   = 112.51 kpc


In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import Planck18 as cosmo   # H0 = 67.66 km/s/Mpc

# ------------------------------------------------------------
# 1. Coordinates (ICRS)
# ------------------------------------------------------------
#gal = SkyCoord(ra=150.829758*u.deg, dec=68.733727*u.deg)
#los = SkyCoord(ra=134.720500*u.deg, dec=73.490833*u.deg)
	
los_ra, los_dec = "169.9835417", "70.67622222"
gal_clust_ras = [
"169.87289",
"170.6479",
"170.96191",
"169.87859",
]

gal_clust_decs = [
"70.91131",
"70.91058",
"70.69499",
"70.34475",
]

gal_clust_zs = [0.2192, 0.2236, 0.2191, 0.1825]

los = SkyCoord(ra=los_ra, dec=los_dec, unit='deg', frame='icrs')

for gal_ra, gal_dec, z_gal in zip(gal_clust_ras, gal_clust_decs, gal_clust_zs):
    gal = SkyCoord(ra=gal_ra, dec=gal_dec, unit='deg', frame='icrs')
    
    theta = gal.separation(los)            # Angle object (38.78°)

    D_A = cosmo.angular_diameter_distance(z_gal)   # ≈ 44.3 kpc

    with u.set_enabled_equivalencies(u.dimensionless_angles()):
        b = (theta * D_A).to(u.kpc)

        #print(f"θ  = {theta.to(u.arcmin):.4f}")
        #print(f"D_A = {D_A.to(u.kpc):.3f}")
        print(f"b   = {b:.2f}")
        #print("--------------------------------")


b   = 3133.00 kpc
b   = 4284.68 kpc
b   = 4267.39 kpc
b   = 3804.00 kpc


In [17]:
import glob
from astropy.io import fits
from astropy.coordinates import SkyCoord
import numpy as np

# List of target coordinates (RA, DEC in sexagesimal strings)
targets = [
    ("20h40m47.886s", "+72d52m56.378s"),
    ("08h58m52.92s",  "+73d29m27.0s"),
    ("21h12m10.760s", "+72d49m38.20s"),
    ("04h45m38.64s",  "+70d18m26.6s"),
    ("21h00m31.09s",  "+72d02m15.22s"),
    ("11h51m07.52s",  "+71d41m44.3s"),
    ("05h52m45.12s",  "+74d12m01.7s"),
    ("20h20m08.92s",  "+70d47m33.96s"),
    ("02h39m03.96s",  "+71d01m04.3s"),
    ("20h50m28.59s",  "+73d54m00.0s"),
    ("11h19m56.05s",  "+70d40m34.4s"),
    ("22h23m53.94s",  "+73d01m33.26s"),
]

# Convert targets to SkyCoord objects
sky_targets = [SkyCoord(ra, dec, unit=("hourangle", "deg")) for ra, dec in targets]

# Find all FITS files in the directory
fits_files = glob.glob("*.fits")

results = {i: [] for i in range(len(targets))}

for fname in fits_files:
    with fits.open(fname) as hdul:
        ra = hdul[1].data['RA']
        dec = hdul[1].data['DEC']
        ra_min, ra_max = np.min(ra), np.max(ra)
        dec_min, dec_max = np.min(dec), np.max(dec)
        # Check each target
        for i, t in enumerate(sky_targets):
            if (ra_min <= t.ra.deg <= ra_max) and (dec_min <= t.dec.deg <= dec_max):
                results[i].append(fname)

# Print results
for i, files in results.items():
    print(f"Target {i+1} ({targets[i][0]}, {targets[i][1]}):")
    if files:
        for f in files:
            print(f"  - {f}")
    else:
        print("  No file covers this target.")

Target 1 (20h40m47.886s, +72d52m56.378s):
  No file covers this target.
Target 2 (08h58m52.92s, +73d29m27.0s):
  No file covers this target.
Target 3 (21h12m10.760s, +72d49m38.20s):
  No file covers this target.
Target 4 (04h45m38.64s, +70d18m26.6s):
  No file covers this target.
Target 5 (21h00m31.09s, +72d02m15.22s):
  No file covers this target.
Target 6 (11h51m07.52s, +71d41m44.3s):
  No file covers this target.
Target 7 (05h52m45.12s, +74d12m01.7s):
  No file covers this target.
Target 8 (20h20m08.92s, +70d47m33.96s):
  No file covers this target.
Target 9 (02h39m03.96s, +71d01m04.3s):
  No file covers this target.
Target 10 (20h50m28.59s, +73d54m00.0s):
  No file covers this target.
Target 11 (11h19m56.05s, +70d40m34.4s):
  No file covers this target.
Target 12 (22h23m53.94s, +73d01m33.26s):
  No file covers this target.
